In [5]:
import time
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import numpy as np

# Step 1: Load the dataset
print("Fetching MNIST dataset...")
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
y = y.astype(int)

# Step 2: Normalize the dataset
print("Normalizing data...")
X = X / 255.0  # Scale pixel values to [0, 1]

# Step 3: Standardize the features
print("Standardizing features...")
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Step 4: Manually split the data into 7k for training and 3k for testing
X_train, X_test, y_train, y_test = X[:7000], X[7000:10000], y[:7000], y[7000:10000]

# Step 5: Set up the SVM with a polynomial kernel of degree 2
C = 1.0  # Regularization parameter
degree = 2  # Polynomial degree
clf = SVC(kernel='poly', C=C, degree=degree)

# Step 6: Apply 10-fold cross-validation on the training set and track time and accuracy
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # 10-fold cross-validation
accuracies = []
fold_times = []

for fold, (train_index, val_index) in enumerate(kf.split(X_train, y_train)):
    print(f"Processing fold {fold + 1}...")  # Console output indicating the current fold
    start_time = time.time()
    
    # Split the data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    
    # Train the model on the fold
    clf.fit(X_train_fold, y_train_fold)
    
    # Evaluate the model on the validation set
    accuracy = clf.score(X_val_fold, y_val_fold)
    accuracies.append(accuracy)
    
    end_time = time.time()
    fold_time = end_time - start_time
    fold_times.append(fold_time)
    
    print(f"Fold {fold + 1}: Accuracy = {accuracy * 100:.2f}%, Time = {fold_time:.2f} seconds")

# Step 7: Compute mean, variance, and best accuracy from cross-validation
mean_accuracy = np.mean(accuracies)
accuracy_variance = np.var(accuracies)
total_time = np.sum(fold_times)
best_accuracy = np.max(accuracies)

print(f"\nMean accuracy: {mean_accuracy*100:.4f}")
print(f"Accuracy variance: {accuracy_variance*100:.4f}")
print(f"Total time for all folds: {total_time:.2f} seconds")
print(f"Best accuracy: {best_accuracy*100:.4f}")

# Step 8: Evaluate the model on the separate test set
print("\nEvaluating the model on the test set...")
clf.fit(X_train, y_train)  # Train on the entire training set
accuracy = clf.score(X_test, y_test)  # Evaluate on the test set
print(f"Accuracy on the test set: {accuracy*100:.4f} %")


Fetching MNIST dataset...
Normalizing data...
Standardizing features...
Processing fold 1...
Fold 1: Accuracy = 93.86%, Time = 42.48 seconds
Processing fold 2...
Fold 2: Accuracy = 96.00%, Time = 42.58 seconds
Processing fold 3...
Fold 3: Accuracy = 94.71%, Time = 33.67 seconds
Processing fold 4...
Fold 4: Accuracy = 95.29%, Time = 30.42 seconds
Processing fold 5...
Fold 5: Accuracy = 92.86%, Time = 27.63 seconds
Processing fold 6...
Fold 6: Accuracy = 94.43%, Time = 28.35 seconds
Processing fold 7...
Fold 7: Accuracy = 94.57%, Time = 25.12 seconds
Processing fold 8...
Fold 8: Accuracy = 94.00%, Time = 23.35 seconds
Processing fold 9...
Fold 9: Accuracy = 92.86%, Time = 23.13 seconds
Processing fold 10...
Fold 10: Accuracy = 94.43%, Time = 21.94 seconds

Mean accuracy: 94.3000
Accuracy variance: 0.0086
Total time for all folds: 298.68 seconds
Best accuracy: 96.0000

Evaluating the model on the test set...
Accuracy on the test set: 92.6333 %
